## UW Pharmacy Student Self-Care Analysis (spring 1_2)

This notebook is a draft using the "spring 1_2" page in the student comment data of the 2022-2023 SY in the UW SoP. The analysis roughly looks at the determinants of the categories of self-care students chose for this quarter using VADER and NLTK sentiment analysis and Pandas table manipulation.

Note: this notebook relies on uploading a single csv sheet for one quarter, along with adding two additional columns called "Category 1" and "Category 2" which categorize each comment based on the 8 facets of self-care.

### Setup

In [1]:
#import python libraries
import numpy as np
import pandas as pd

!pip install vaderSentiment
!pip install --upgrade pip
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from textblob import TextBlob
import nltk
nltk.download('brown')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

import spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 KB 25.1 MB/s eta 0:00:00
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 63.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebo

In [2]:
# Can change the file csv right here
file_csv = "spring1_2.csv"

data = pd.read_csv(file_csv)

first_column = data.columns[1]
data = data[[first_column, "Category 1", "Category 2"]]

In [3]:
all_dups = pd.DataFrame()
for i in range(len(data)):
    if not pd.isnull(data["Category 2"][i]):
        temp = data[[first_column, "Category 2"]].iloc[i]
        all_dups = all_dups.append(temp)
    else:
        continue
all_dups = all_dups[[first_column, "Category 2"]]
all_dups = all_dups.rename(columns={"Category 2": "Category"})

### Working data

In [4]:
data = data.rename(columns={"Category 1": "Category"})
data = data[[first_column, "Category"]]
data = data.append(all_dups)
data

,Spr 1_2,Category
0,Had a very busy weekend with my sister visitin...,Physical
1,I decided to spend my self care time by going ...,Physical
2,I took the time to finish building my baby Yod...,Mental
3,I started a new Asian drama (tv show) called T...,Mental
4,Yesterday was my birthday and my boyfriend pla...,Community
...,...,...
96,The activity I chose to do was to get lunch wi...,Environmental
98,I am going to watch the cherry blossom on camp...,Community
100,I went to Seward park and took a long walk wit...,Environmental
104,I went on a half hour run to Gas works park th...,Environmental


### Category frequencies within student comments

The comments data is displayed here in terms of each of the 8 self-care categories (physical, mental, community, emotional, environmental, spiritual, and occupational) through a pie chart. 

In [5]:
import plotly.express as px

dfg = data.groupby("Category").count().sort_values(by=first_column, ascending=False)
print("All reflections processed (includes duplicate categories): ", np.sum(dfg[first_column]))

dfg_pie = px.pie(dfg.reset_index(), values='Spr 1_2', names='Category', title='Category Frequencies')
dfg_pie

All reflections processed (includes duplicate categories):  167


### Sentiment analysis (positivity polarity score for each comment)

In this section, I scored each of the comments using the VADER sentiment analysis function. The function sentiment_scores(sentence) will return the positivity polarity score for each comment probabilistically between -1 and 1, with -1 being the most negative and 1 being the most positive. Note that this model is untrained and will be a very rough interpretation of the comments, i.e. not trained on what self-care is as a whole but on the general tone of each comment.

In [6]:
# function to print sentiments
# of the sentence (vader)

#taken from geekforgeeks
def sentiment_scores(sentence):
 
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
 
    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)

    return sentiment_dict['compound']

In [7]:
scores = pd.DataFrame(columns={"Win 1_2", "Score"})
for i in data[first_column]:
    sentence = str(i)
    score = sentiment_scores(sentence)
    scores = scores.append({first_column: sentence, "Score": score}, ignore_index=True)

In [8]:
scores_categories = data.merge(scores, on=first_column, how="right")
scores_categories

,Spr 1_2,Category,Win 1_2,Score
0,Had a very busy weekend with my sister visitin...,Physical,NaN,0.0000
1,I decided to spend my self care time by going ...,Physical,NaN,0.6239
2,I took the time to finish building my baby Yod...,Mental,NaN,0.5859
3,I took the time to finish building my baby Yod...,Emotional,NaN,0.5859
4,I started a new Asian drama (tv show) called T...,Mental,NaN,0.6369
...,...,...,...,...
286,I went to Seward park and took a long walk wit...,Environmental,NaN,0.4939
287,I went on a half hour run to Gas works park th...,Physical,NaN,0.0000
288,I went on a half hour run to Gas works park th...,Environmental,NaN,0.0000
289,"Today for health, wellness, and self-care I pl...",Spiritual,NaN,0.9501


In [9]:
fig = px.scatter(scores_categories, y="Score", x="Category", title="Polarity Scores by Category")
mean_scores = scores_categories.groupby('Category').mean()
for c in mean_scores.index:
    fig.add_scatter(x=[c],
                    y=[mean_scores.loc[c]['Score']],
                    marker=dict(
                        color='red',
                        size=10
                    ),
                name=f'{c} mean')

fig.show()

In [10]:
px.bar(mean_scores.reset_index().sort_values(by="Score", ascending=False), x="Category", y="Score", title="Average Scores per Category")

### Scores of each category 

In [11]:
print('Physical category table')
scores_categories_physical = scores_categories[scores_categories["Category"] == "Physical"]
scores_categories_physical

Physical category table


,Spr 1_2,Category,Win 1_2,Score
0,Had a very busy weekend with my sister visitin...,Physical,NaN,0.0000
1,I decided to spend my self care time by going ...,Physical,NaN,0.6239
6,Resilience Coping Skills - I woke up and ate a...,Physical,NaN,0.6901
17,This week I cooked some Korean inspired ground...,Physical,NaN,0.4939
18,Went on a 30 minute walk,Physical,NaN,0.0000
...,...,...,...,...
260,"For this week, I was able to visit home in San...",Physical,NaN,0.6467
271,I went to the gym.,Physical,NaN,0.0000
277,I went to the gym this morning and cooked a re...,Physical,NaN,0.4927
280,My self care for the week was making dinner fo...,Physical,NaN,0.8689


In [12]:
print('Mental category table')
scores_categories_mental = scores_categories[scores_categories["Category"] == "Mental"]
scores_categories_mental

Mental category table


,Spr 1_2,Category,Win 1_2,Score
2,I took the time to finish building my baby Yod...,Mental,NaN,0.5859
4,I started a new Asian drama (tv show) called T...,Mental,NaN,0.6369
9,I cleaned my room,Mental,NaN,0.0000
12,I spent Saturday night with my best friend fro...,Mental,NaN,0.9371
15,For my self wellness my friend Katy and I went...,Mental,NaN,0.8622
16,"Today I spent some time reading ""The Vanishing...",Mental,NaN,0.5093
21,I got a new air-fryer so I spent some time res...,Mental,NaN,0.0000
29,I am spending 2 hours to practice dancing for ...,Mental,NaN,0.0000
33,reached out to a friend and then we played a g...,Mental,NaN,0.7184
37,"For wellness, I organized and decluttered my c...",Mental,NaN,0.9757


In [13]:
print('Community category table')
scores_categories_community = scores_categories[scores_categories["Category"] == "Community"]
scores_categories_community

Community category table


,Spr 1_2,Category,Win 1_2,Score
5,Yesterday was my birthday and my boyfriend pla...,Community,NaN,0.9757
10,Hung out with my friends,Community,NaN,0.4767
11,I spent Saturday night with my best friend fro...,Community,NaN,0.9371
14,For my self wellness my friend Katy and I went...,Community,NaN,0.8622
32,reached out to a friend and then we played a g...,Community,NaN,0.7184
42,I spoke with my brother over the phone and we ...,Community,NaN,0.0000
45,I took a long walk with my husband.,Community,NaN,0.0000
48,I went to the Flower World and the Molback’s N...,Community,NaN,0.0000
51,I went to lunch and then on a walk with a frie...,Community,NaN,0.4939
67,I went to get to get coffee in Capitol Hill wi...,Community,NaN,0.0000


In [14]:
print('Emotional category table')
scores_categories_emotional = scores_categories[scores_categories["Category"] == "Emotional"]
scores_categories_emotional

Emotional category table


,Spr 1_2,Category,Win 1_2,Score
3,I took the time to finish building my baby Yod...,Emotional,NaN,0.5859
43,I’m questioning myself How am I feeling? What ...,Emotional,NaN,0.9908
97,Today I am working on my physical and emotiona...,Emotional,NaN,0.8720
99,"On March 30, I listened to relaxing music whil...",Emotional,NaN,0.4939
166,"Today for health, wellness, and self-care I pl...",Emotional,NaN,0.9501
170,I took the time to finish building my baby Yod...,Emotional,NaN,0.5859
195,I’m questioning myself How am I feeling? What ...,Emotional,NaN,0.9908
236,Today I am working on my physical and emotiona...,Emotional,NaN,0.8720
290,"Today for health, wellness, and self-care I pl...",Emotional,NaN,0.9501


In [15]:
print('Spiritual category table')
scores_categories_spiritual = scores_categories[scores_categories["Category"] == "Spiritual"]
scores_categories_spiritual

Spiritual category table


,Spr 1_2,Category,Win 1_2,Score
35,"I have been going on walks, because it calms m...",Spiritual,NaN,0.7935
114,Today I used the 20 minute break to do a short...,Spiritual,NaN,0.0000
129,Today is day 5 out of 30 for Ramadan so I plan...,Spiritual,NaN,0.5719
165,"Today for health, wellness, and self-care I pl...",Spiritual,NaN,0.9501
248,Today I used the 20 minute break to do a short...,Spiritual,NaN,0.0000
261,Today is day 5 out of 30 for Ramadan so I plan...,Spiritual,NaN,0.5719
289,"Today for health, wellness, and self-care I pl...",Spiritual,NaN,0.9501


In [16]:
print('Occupational category table')
scores_categories_occupational = scores_categories[scores_categories["Category"] == "Occupational"]
scores_categories_occupational

Occupational category table


,Spr 1_2,Category,Win 1_2,Score
95,I reached out to a colleague to help me study ...,Occupational,NaN,0.4767
125,I went to my first annual health and wellness ...,Occupational,NaN,0.4404
234,I reached out to a colleague to help me study ...,Occupational,NaN,0.4767


### Frequency of nouns and verbs 

In [17]:
!pip install spacy -q
!python -m spacy download en_core_web_sm -q

from collections import Counter
import en_core_web_sm

2023-06-01 17:38:20.249354: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 17:38:20.442272: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-01 17:38:20.442310: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-01 17:38:20.473288: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-01 17:38:21.740185: W tensorflow/stream_executor/pla

Note: Just for the sake of experimentation, I only did this part on comments categorized as "physical." Can easily broaden it to more categories if useful.

This section takes the top 100 ('top' meaning most frequent) words from a particular category along with its count within the comments. Again, note that there may be duplicates depending on if the comment was repeated if it was categorized twice. I then created a function to extract nouns and verbs within that category. This can potentially be helpful if we were to see which action within each category students tended to choose.

In [18]:
def fig_frequencies(tbl, keyword):
    counter = Counter(" ".join(tbl[first_column]).split()).most_common(100)
    array = []
    for c in counter:
        array = np.append(array, c[0])

    sentence = ','.join(array)

    def extract_nouns_verbs(sentence):
        text = nltk.word_tokenize(sentence)
        pos_tagged = nltk.pos_tag(text)
        nouns_verbs = filter(lambda x:x[1]=='NN' or x[1] == 'VB',pos_tagged)
        return list(nouns_verbs)

    nouns_verbs = pd.DataFrame(extract_nouns_verbs(sentence), columns=['Word', 'Word type'])
    counted_words = pd.DataFrame(counter, columns=['Word', 'Word frequency'])

    noun_verb_frequency = nouns_verbs.merge(counted_words, on='Word', how='left').sort_values(by='Word frequency', ascending=False)

    nlp = en_core_web_sm.load()
    def all_nouns(sentence):
        doc = nlp(sentence)
        nouns = [(token.lemma_, "NN") for token in doc if token.pos_ == "NOUN"]
        return nouns

    def all_verbs(sentence):
        doc = nlp(sentence)
        verbs = [(token.lemma_, "VB") for token in doc if token.pos_ == "VERB"]
        return verbs

    nouns_verbs_spacy = all_nouns(sentence) + all_verbs(sentence)

    nouns_verbs_sp = pd.DataFrame(nouns_verbs_spacy, columns=['Word', 'Word type'])
    counted_words = pd.DataFrame(counter, columns=['Word', 'Word frequency'])

    noun_verb_frequency_sp = nouns_verbs_sp.merge(counted_words, on='Word', how='inner').sort_values(by='Word frequency', ascending=False)
    noun_verb_frequency_sp = noun_verb_frequency_sp.drop_duplicates(subset='Word', keep="last")

    fig = px.bar(noun_verb_frequency_sp, x='Word', y='Word frequency', color='Word type', title='Noun/verb frequencies for Category: ' + keyword)
    fig.update_layout(xaxis_categoryorder = 'total descending')

    fig.show()

In [19]:
fig_frequencies(scores_categories_physical, 'Physical');

/shared-libs/python3.9/py/lib/python3.9/site-packages/spacy/language.py:1895: UserWarning:

[W123] Argument disable with value [] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.



In [20]:
fig_frequencies(scores_categories_mental, 'Mental');

In [21]:
fig_frequencies(scores_categories_emotional, 'Emotional');

In [22]:
fig_frequencies(scores_categories_community, 'Community');

In [23]:
fig_frequencies(scores_categories_spiritual, 'Spiritual');

In [24]:
fig_frequencies(scores_categories_occupational, 'Occupational');

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=443b2d8b-ed93-43a6-bbe3-c3fe69ac80e1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>